<a href="https://colab.research.google.com/github/gyuholee94/NLP/blob/master/NLP_%EC%97%B0%EC%8A%B5_4_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


In [5]:
for item in imdb['train']:
  print(item)
  break

(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


In [7]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []


for s, l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())


for s, l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)



In [35]:
vocab_size = 10000
embedding_dim = 16
max_len = 120
trunc_type = 'post'
oov_tok = '<OOV>'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_len, truncating=trunc_type)     
                                  

In [36]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_len)

In [37]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [38]:
reverse_word_index[3]

'and'

In [39]:
len(word_index)

88583

In [40]:
def decode_review(text):
  return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [41]:
print(decode_review(padded[3]))#패딩된건 사전에 없으니까(인덱스에 없어서)물음표로 표시됨. 

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received


In [42]:
print(training_sentences[3])

This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [48]:
import tensorflow as tf
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_len),#문장을 받아와서 임베딩 해주는 애
                             tf.keras.layers.GlobalAveragePooling1D(),#flatten이랑 똑같은 애
                             tf.keras.layers.Dense(6, activation = 'relu'),
                             tf.keras.layers.Dense(1, activation = 'sigmoid')

])


In [49]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 102       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [50]:
temp_tk = Tokenizer()
temp_tk.fit_on_texts(training_sentences)
wi = temp_tk.word_index
len(wi)

88582

In [51]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [52]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.5484 - accuracy: 0.7383 - val_loss: 0.3815 - val_accuracy: 0.8408
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.3284 - accuracy: 0.8632 - val_loss: 0.3327 - val_accuracy: 0.8588
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2670 - accuracy: 0.8917 - val_loss: 0.3297 - val_accuracy: 0.8582
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2310 - accuracy: 0.9117 - val_loss: 0.3497 - val_accuracy: 0.8526
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2054 - accuracy: 0.9236 - val_loss: 0.3668 - val_accuracy: 0.8478
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1841 - accuracy: 0.9333 - val_loss: 0.3930 - val_accuracy: 0.8427
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1670 - accuracy: 0.9399 - val_loss: 0.4242 - val_accuracy: 0.8389
Epoch 

In [53]:
e = model.layers[0]

In [57]:
weights = e.get_weights()[0]

In [58]:
weights.shape

(10000, 16)

In [59]:
import io
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + '\n')
  out_v.write('\t'.join([str(x) for x in embeddings])+'\n')
out_v.close()
out_m.close()

In [61]:
from google.colab import files
files.download('vecs.tsv')
files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>